In [6]:
from pathlib import Path
from collections import Counter

# ====== 경로 설정 ======
LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data_for_study")

# YOLO 클래스 이름 매핑
classes = [
    "Divot",        # 0
    "Fixed_Divot",  # 1
    "Dieased_Grass",# 2
    "Pole",         # 3
    "Obstacle",     # 4
    "Sprinkler",    # 5
    "Drain",        # 6
    "Golf ball"     # 7
]

# ====== 객체 카운트 ======
counter = Counter()

# txt 라벨 파일 순회
for txt_file in LABEL_DIR.glob("*.txt"):
    with open(txt_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls_id = int(parts[0])
            counter[cls_id] += 1

# ====== 결과 출력 ======
print("=== 클래스별 객체 개수 ===")
for i, cls_name in enumerate(classes):
    print(f"{cls_name:<15}: {counter.get(i, 0)}")

print("\n총 객체 수:", sum(counter.values()))


=== 클래스별 객체 개수 ===
Divot          : 192
Fixed_Divot    : 186
Dieased_Grass  : 67
Pole           : 22
Obstacle       : 0
Sprinkler      : 10
Drain          : 24
Golf ball      : 20

총 객체 수: 521


In [7]:
# augment_balance_final_custom.py
# - 사용자가 원하는 TARGET(클래스별 최종 객체 수)을 직접 설정
# - 재사용 제한을 클래스별로 가산할 수 있도록 개선
# - 나머지 안전장치(yolo_sanitize, 빈/이상 라인 제거, 로그 등) 유지

from pathlib import Path
from collections import Counter, defaultdict
import random, csv
import cv2
import albumentations as A

# ========== 사용자 설정 ==========
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data_for_study")
OUT_DIR = BASE / "aug_balanced"     # 단일 폴더 (이미지/라벨 저장)
IMG_EXTS = [".jpg", ".jpeg", ".png"]
SEED = 42
random.seed(SEED)

# (필수) 최종 목표 개수를 직접 지정하세요. (원본처럼!!)
# 예시: Divot 3500, Fixed_Divot 3500, Dieased_Grass 300, 나머지는 기존 유지 or 적절히 설정
# 클래스 매핑: 0 Divot, 1 Fixed_Divot, 2 Dieased_Grass, 3 Pole, 4 Obstacle, 5 Sprinkler, 6 Drain, 7 Golf ball
TARGET = {
    0: 400,  # Divot
    1: 400,  # Fixed_Divot
    2: 150,   # Dieased_Grass
    3: 50,   # Pole (예시)
    4: 0,   # Obstacle (예시)
    5: 30,   # Sprinkler (예시)
    6: 50,   # Drain (예시)
    7: 50,   # Golf ball (예시)
}
ALL_CLASS_IDS = list(range(8))
ALLOW_IDS = set(ALL_CLASS_IDS)

# 재사용 제한 정책
# - 기본 회수
MAX_USES_BASE = 2
# - 클래스별 가산치(해당 클래스를 포함하고 "아직 부족"한 경우에만 적용)
#   예: 2와 3이 특히 부족하면 2회 더 허용, 5는 1회 더 허용… 필요에 맞게 수정
MAX_USES_BOOST_PER_CLASS = {
    0: 3,
    1: 3,
    2: 5,   # Dieased_Grass 부족하면 +2
    3: 5,
    # 4: 5,
    5: 5,   # Pole 부족하면 +2
    6: 5,   # Sprinkler 부족하면 +1
    7: 5,
    # 필요시 0..7 아무거나 추가/수정
}

# 배경(라벨 없는 이미지) 증강 배수
# 값=2 → 원본 1 + 증강 2 = 총 3배
BG_AUG_MULTIPLIER = 2

# 증강 파이프라인(색/조명 중심, 약한 변형)
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.12, 0.18),
                               contrast_limit=(-0.12, 0.18), p=0.9),
    A.HueSaturationValue(hue_shift_limit=2, sat_shift_limit=12, val_shift_limit=10, p=0.8),
    A.RGBShift(r_shift_limit=8, g_shift_limit=6, b_shift_limit=6, p=0.5),
    A.RandomGamma(gamma_limit=(90, 120), p=0.4),
    A.OneOf([
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
        A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.08, 0.25), p=0.5),
        A.NoOp()
    ], p=0.4),
], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"], min_visibility=0.25))

# ========== 유틸 ==========
def ensure_out_dir():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

def read_yolo_label(lbl: Path):
    boxes, cls = [], []
    if not lbl.exists():
        return boxes, cls
    with open(lbl, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 5:
                continue
            try:
                c = int(float(p[0]))
            except Exception:
                continue
            try:
                x, y, w, h = map(float, p[1:5])
            except Exception:
                continue
            x = min(max(x, 0.0), 1.0)
            y = min(max(y, 0.0), 1.0)
            w = min(max(w, 0.0), 1.0)
            h = min(max(h, 0.0), 1.0)
            if w <= 0.0 or h <= 0.0:
                continue
            boxes.append([x, y, w, h]); cls.append(c)
    return boxes, cls

def write_yolo_label(lbl_path: Path, boxes, cls, *, allow_ids=ALLOW_IDS):
    lbl_path.parent.mkdir(parents=True, exist_ok=True)
    with open(lbl_path, "w") as f:
        for c, (x, y, w, h) in zip(cls, boxes):
            try:
                c_int = int(float(c))
            except Exception:
                continue
            if allow_ids is not None and c_int not in allow_ids:
                continue
            x = float(max(0.0, min(1.0, x)))
            y = float(max(0.0, min(1.0, y)))
            w = float(max(0.0, min(1.0, w)))
            h = float(max(0.0, min(1.0, h)))
            if w <= 0.0 or h <= 0.0:
                continue
            f.write(f"{c_int} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

def yolo_sanitize(boxes):
    eps = 1e-6
    out = []
    for x, y, w, h in boxes:
        l = x - w/2.0; r = x + w/2.0
        t = y - h/2.0; b = y + h/2.0
        l = max(0.0, min(1.0, l))
        r = max(0.0, min(1.0, r))
        t = max(0.0, min(1.0, t))
        b = max(0.0, min(1.0, b))
        if r - l <= eps or b - t <= eps:
            continue
        nx = (l + r) / 2.0
        ny = (t + b) / 2.0
        nw = (r - l)
        nh = (b - t)
        nx = min(max(nx, eps), 1.0 - eps)
        ny = min(max(ny, eps), 1.0 - eps)
        nw = min(max(nw, eps), 1.0 - eps)
        nh = min(max(nh, eps), 1.0 - eps)
        out.append([nx, ny, nw, nh])
    return out

def find_image_for_label(lbl: Path):
    stem = lbl.stem
    for ext in IMG_EXTS:
        p = lbl.with_suffix(ext)
        if p.exists(): return p
    for parent in [lbl.parent, *lbl.parents]:
        img_dir = parent / "images"
        if img_dir.exists():
            for ext in IMG_EXTS:
                p = img_dir / f"{stem}{ext}"
                if p.exists(): return p
    for ext in IMG_EXTS:
        hits = list(BASE.rglob(f"{stem}{ext}"))
        if hits: return hits[0]
    return None

def current_counts(root: Path):
    cnt = Counter()
    for lbl in root.rglob("*.txt"):
        _, cls = read_yolo_label(lbl)
        cnt.update(cls)
    return cnt

def collect_background_images(base: Path):
    label_stems = {p.stem for p in base.rglob("*.txt")}
    bg_images = []
    for ext in IMG_EXTS:
        for img in base.rglob(f"*{ext}"):
            if img.stem not in label_stems:
                bg_images.append(img)
    return sorted(bg_images)

def load_image(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        raise RuntimeError(f"Failed to read image: {path}")
    return img

def next_unique_name(stem: str, used: set, tag: str):
    i = 1
    while True:
        s = f"{stem}_{tag}{i:04d}"
        if s not in used:
            used.add(s)
            return s
        i += 1

def effective_cap_for_file(file_cls_counter: Counter, deficit: Counter):
    """
    이 파일(라벨)의 클래스 구성과 현재 부족분을 보고
    MAX_USES_BASE + (부족 클래스 중 최대 가산치) 를 반환.
    """
    cap = MAX_USES_BASE
    best_boost = 0
    for c, n in file_cls_counter.items():
        if n <= 0: 
            continue
        if deficit.get(c, 0) > 0:
            boost = MAX_USES_BOOST_PER_CLASS.get(c, 0)
            if boost > best_boost:
                best_boost = boost
    return cap + best_boost

# ========== 실행 ==========
ensure_out_dir()

# 1) 배경 3배(원본 + 증강2)
bg_list = collect_background_images(BASE)
print(f"Background images: {len(bg_list)}")

used_names = set()
for bg_img in bg_list:
    try:
        img = load_image(bg_img)
    except Exception:
        continue
    for _ in range(BG_AUG_MULTIPLIER):
        out_stem = next_unique_name(bg_img.stem, used_names, tag="bg")
        out_img = OUT_DIR / f"{out_stem}{bg_img.suffix.lower()}"
        out_lbl = OUT_DIR / f"{out_stem}.txt"
        transformed = aug(image=img, bboxes=[], class_labels=[])
        aug_img = transformed["image"]
        cv2.imwrite(str(out_img), aug_img)
        with open(out_lbl, "w") as f:
            pass
print("Background augmentation done.")

# 2) 라벨 쌍 후보
label_files = sorted(BASE.rglob("*.txt"))
candidates = []  # (img_path, lbl_path, file_counts, stem)
skipped = 0
for lbl in label_files:
    boxes, cls_list = read_yolo_label(lbl)
    if not cls_list:
        continue
    img = find_image_for_label(lbl)
    if img is None or not img.exists():
        skipped += 1
        continue
    fcnt = Counter(cls_list)
    candidates.append((img, lbl, fcnt, lbl.stem))
if not candidates:
    raise SystemExit("No labeled image-label pairs found.")
print(f"Labeled pairs: {len(candidates)}, skipped(no image match): {skipped}")

# 3) 초기 카운트(원본)
base_cnt = current_counts(BASE)
print("Base counts:", dict(base_cnt))

# 4) 타깃/결손 세팅
cur = Counter(base_cnt)
# TARGET이 현재보다 작은 경우는 결손 0으로 간주(감소 작업은 하지 않음)
deficit = Counter({c: max(0, TARGET.get(c, 0) - cur.get(c, 0)) for c in ALL_CLASS_IDS})

use_count = defaultdict(int)
log_path = OUT_DIR / "log.csv"
with open(log_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["src_img","src_lbl","out_img","out_lbl","add_per_class","totals_after"])

MAX_ITERS = 200000
iters = 0

def score(deficit: Counter, fcnt: Counter):
    # 남은 결손에 기여도가 큰 파일 우선
    return sum(deficit[c] * fcnt.get(c, 0) for c in ALL_CLASS_IDS if deficit[c] > 0)

while any(deficit[c] > 0 for c in ALL_CLASS_IDS) and iters < MAX_ITERS:
    iters += 1
    best = None
    best_score = 0

    for img, lbl, fcnt, stem in candidates:
        cap = effective_cap_for_file(fcnt, deficit)
        if use_count[img] >= cap:
            continue
        sc = score(deficit, fcnt)
        if sc > best_score:
            best_score = sc
            best = (img, lbl, fcnt, stem)

    if not best or best_score <= 0:
        print("No more useful candidates for remaining deficits. Stopping.")
        break

    img_path, lbl_path, fcnt, stem = best
    out_stem = next_unique_name(stem, used_names, tag="dup")
    out_img = OUT_DIR / f"{out_stem}{img_path.suffix.lower()}"
    out_lbl = OUT_DIR / f"{out_stem}.txt"

    # 증강
    img = load_image(img_path)
    boxes0, cls0 = read_yolo_label(lbl_path)
    boxes0 = yolo_sanitize(boxes0)

    transformed = aug(image=img, bboxes=boxes0, class_labels=cls0)
    aug_img = transformed["image"]
    aug_boxes = transformed["bboxes"]
    aug_cls = transformed["class_labels"]

    keep_boxes, keep_cls = [], []
    for (x, y, w, h), c in zip(aug_boxes, aug_cls):
        if w > 0 and h > 0:
            keep_boxes.append([float(x), float(y), float(w), float(h)])
            keep_cls.append(int(float(c)))

    # 실제로 부족 클래스에 기여하는지 확인
    test_cnt = Counter(keep_cls)
    if sum(test_cnt[c] for c in ALL_CLASS_IDS if deficit[c] > 0) <= 0 or not keep_cls:
        use_count[img_path] += 1
        continue

    cv2.imwrite(str(out_img), aug_img)
    write_yolo_label(out_lbl, keep_boxes, keep_cls, allow_ids=ALLOW_IDS)

    _, saved_cls = read_yolo_label(out_lbl)
    add_cnt = Counter(saved_cls)

    cur.update(add_cnt)
    for c in ALL_CLASS_IDS:
        deficit[c] = max(0, TARGET.get(c, 0) - cur.get(c, 0))
    use_count[img_path] += 1

    add_str = "{" + ", ".join(f"{k}:{add_cnt.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
    tot_str = "{" + ", ".join(f"{k}:{cur.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
    with open(log_path, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow([str(img_path), str(lbl_path), str(out_img), str(out_lbl), add_str, tot_str])

    if iters % 50 == 0 or all(deficit[c]==0 for c in ALL_CLASS_IDS):
        print(f"[{iters}] cur={dict(cur)} deficit={dict(deficit)} last={out_stem}")

print("\n=== Finished ===")
print("Final counts:", dict(cur))
print("Deficit:", dict(deficit))
print(f"Saved to: {OUT_DIR}")
print(f"Log: {log_path}")


Background images: 0
Background augmentation done.
Labeled pairs: 224, skipped(no image match): 0
Base counts: {6: 24, 2: 67, 0: 192, 7: 20, 1: 186, 5: 10, 3: 22}
[50] cur={6: 24, 2: 77, 0: 311, 7: 20, 1: 317, 5: 10, 3: 22} deficit={0: 89, 1: 83, 2: 73, 3: 28, 4: 0, 5: 20, 6: 26, 7: 30} last=20250904_01602_dup0004
[100] cur={6: 28, 2: 141, 0: 379, 7: 22, 1: 394, 5: 10, 3: 24} deficit={0: 21, 1: 6, 2: 9, 3: 26, 4: 0, 5: 20, 6: 22, 7: 28} last=20250904_04097_dup0002
[150] cur={6: 39, 2: 150, 0: 395, 7: 44, 1: 416, 5: 19, 3: 46} deficit={0: 5, 1: 0, 2: 0, 3: 4, 4: 0, 5: 11, 6: 11, 7: 6} last=20250904_06103_dup0002
[181] cur={6: 50, 2: 152, 0: 401, 7: 50, 1: 425, 5: 30, 3: 50} deficit={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0} last=20250904_06106_dup0006

=== Finished ===
Final counts: {6: 50, 2: 152, 0: 401, 7: 50, 1: 425, 5: 30, 3: 50}
Deficit: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
Saved to: /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data_for_study/

In [8]:
# split_iter_strat_with_bg_8cls.py
from pathlib import Path
from collections import Counter, defaultdict
import random, shutil

# ====== 설정 ======
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data_for_study")
DST = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data_for_study_split")

# 이미지 확장자
IMG_EXTS = [".jpg", ".jpeg", ".png"]

# 분할 비율 (test는 나중에 쓸 수 있도록 주석만 남김)
RATIOS = {
    "train": 0.80,
    "val":   0.20,
    # "test":  0.00,  # ← 필요해지면 주석 해제 + 비율 조정
}

# 클래스 ID (0~7)
# 0: Divot, 1: Fixed_Divot, 2: Dieased_Grass, 3: Pole,
# 4: Obstacle, 5: Sprinkler, 6: Drain, 7: Golf ball
LABELS = list(range(8))

# 배경 표기
BG = "bg"

SEED = 42
random.seed(SEED)

# ====== 유틸 ======
def ensure_dirs():
    for s in RATIOS:
        (DST / "images" / s).mkdir(parents=True, exist_ok=True)
        (DST / "labels" / s).mkdir(parents=True, exist_ok=True)

def find_image(stem: str):
    for ext in IMG_EXTS:
        p = SRC / f"{stem}{ext}"
        if p.exists():
            return p
    # 하위 폴더까지 탐색 (파일명이 유니크하다는 가정)
    for ext in IMG_EXTS:
        hits = list(SRC.rglob(f"{stem}{ext}"))
        if hits:
            return hits[0]
    return None

def parse_label_file(p: Path):
    """해당 파일에 등장한 '클래스 집합'을 반환. 비어있으면 빈 set."""
    labs = set()
    try:
        with open(p, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    try:
                        cid = int(float(parts[0]))
                    except:
                        continue
                    if cid in LABELS:
                        labs.add(cid)
    except:
        pass
    return labs

# ====== 1) 수집: (stem, img_path, lbl_path or None, labels_set) ======
items = []
stems = set()

# 상위/하위 폴더 전체에서 이미지 수집
for ext in IMG_EXTS:
    for img in SRC.rglob(f"*{ext}"):
        stems.add(img.stem)

for stem in sorted(stems):
    img = find_image(stem)
    if img is None:
        continue
    lbl = img.with_suffix(".txt")
    if not lbl.exists():
        # 라벨 파일이 없으면 배경으로 취급
        items.append((stem, img, None, {BG}))
    else:
        labs = parse_label_file(lbl)
        if labs:
            items.append((stem, img, lbl, labs))
        else:
            # 빈 라벨 파일이면 배경으로 취급 (YOLOv8은 .txt 없어도 배경 처리하므로 복사 시 라벨 생략)
            items.append((stem, img, None, {BG}))

if not items:
    raise SystemExit("No images found in source folder.")

all_labels = LABELS + [BG]
N = len(items)

# ====== 2) 목표치 계산 (샘플 개수 기준) ======
target_sizes = {s: int(round(N * r)) for s, r in RATIOS.items()}

# 합 보정(반올림으로 생긴 오차 보정)
while sum(target_sizes.values()) < N:
    k = min(target_sizes, key=lambda s: target_sizes[s])
    target_sizes[k] += 1
while sum(target_sizes.values()) > N:
    k = max(target_sizes, key=lambda s: target_sizes[s])
    target_sizes[k] -= 1

# 전체 라벨 분포 집계 (멀티라벨: 이미지 하나가 여러 라벨을 가질 수 있음)
global_counts = Counter()
for _, _, _, labs in items:
    for l in labs:
        global_counts[l] += 1

# 분할별 라벨 목표(비율 유지)
label_targets = {s: {l: global_counts[l]*RATIOS[s] for l in all_labels} for s in RATIOS}

# ====== 3) Iterative stratification ======
# 각 라벨이 포함된 샘플 인덱스 집합
idxs_by_label = {l: set() for l in all_labels}
for i, it in enumerate(items):
    for l in it[3]:
        idxs_by_label[l].add(i)

assigned = {s: [] for s in RATIOS}
split_sizes = {s: 0 for s in RATIOS}
split_label_counts = {s: Counter() for s in RATIOS}
unassigned = set(range(N))

def cap_ok(s):
    return split_sizes[s] < target_sizes[s]

def need(s, l):
    return label_targets[s][l] - split_label_counts[s][l]

def pick_label():
    # 남은 필요량이 큰 라벨 + 희소성 가중치 고려
    best_l, best_score = None, -1e18
    for l in all_labels:
        if not (idxs_by_label[l] & unassigned):
            continue
        pos_need_sum = sum(max(0.0, need(s, l)) for s in RATIOS if cap_ok(s))
        rarity_w = 1.0 / max(1.0, global_counts[l])  # 희소할수록 가중↑
        score = pos_need_sum * rarity_w
        if score > best_score:
            best_score, best_l = score, l
    return best_l

def assign_idx_to_best_split(i):
    labs = items[i][3]
    best_s, best_cost = None, 1e18
    for s in RATIOS:
        if not cap_ok(s):
            continue
        # 라벨 오차 변화 + 사이즈 편차(가벼운 가중치)
        label_term = 0.0
        for l in labs:
            cur = split_label_counts[s][l]
            tgt = label_targets[s][l]
            label_term += abs((cur+1) - tgt) - abs(cur - tgt)
        size_term = abs((split_sizes[s]+1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
        cost = label_term + 0.05*size_term
        if cost < best_cost:
            best_cost, best_s = cost, s
    if best_s is None:
        return False
    assigned[best_s].append(i)
    split_sizes[best_s] += 1
    for l in labs:
        split_label_counts[best_s][l] += 1
    unassigned.discard(i)
    for l in labs:
        idxs_by_label[l].discard(i)
    return True

# 메인 루프
while unassigned:
    l = pick_label()
    if l is None:
        break
    pool = list(idxs_by_label[l] & unassigned)
    random.shuffle(pool)

    # l 라벨이 더 필요한 split 순으로 채우기
    splits_by_need = sorted([s for s in RATIOS if cap_ok(s)], key=lambda s: need(s, l), reverse=True)
    progressed = False
    for s in splits_by_need:
        best_i, best_cost = None, 1e18
        for i in pool:
            labs = items[i][3]
            label_term = 0.0
            for ll in labs:
                cur = split_label_counts[s][ll]
                tgt = label_targets[s][ll]
                label_term += abs((cur+1) - tgt) - abs(cur - tgt)
            size_term = abs((split_sizes[s]+1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
            cost = label_term + 0.05*size_term
            if cost < best_cost:
                best_cost, best_i = cost, i
        if best_i is not None:
            assigned[s].append(best_i)
            split_sizes[s] += 1
            for ll in items[best_i][3]:
                split_label_counts[s][ll] += 1
            unassigned.discard(best_i)
            for ll in items[best_i][3]:
                idxs_by_label[ll].discard(best_i)
            pool.remove(best_i)
            progressed = True
            if not cap_ok(s):
                continue
    if not progressed:
        break

# 남은 샘플은 사이즈/라벨 오차 최소 split로 배정
if unassigned:
    rest = list(unassigned)
    random.shuffle(rest)
    for i in rest:
        assign_idx_to_best_split(i)

# ====== 4) 복사 ======
ensure_dirs()

def copy_pair(idx, split):
    stem, img, lbl, labs = items[idx]
    dst_img = DST / "images" / split / img.name
    dst_lbl = DST / "labels" / split / f"{stem}.txt"
    shutil.copy2(img, dst_img)
    if lbl and lbl.exists():
        shutil.copy2(lbl, dst_lbl)
    # bg는 라벨 파일 생성 안 함 (YOLOv8이 없는 경우를 배경으로 처리)

for s, idxs in assigned.items():
    for i in idxs:
        copy_pair(i, s)

# ====== 5) data.yaml ======
with open(DST / "data.yaml", "w") as f:
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    # if "test" in RATIOS and target_sizes.get("test", 0) > 0:
    #     f.write("test: images/test\n")
    f.write("\n")
    f.write("nc: 8\n")
    f.write('names: ["Divot","Fixed_Divot","Dieased_Grass","Pole","Obstacle","Sprinkler","Drain","Golf ball"]\n')

# ====== 6) 리포트 ======
def summarize():
    rep = []
    total = sum(len(v) for v in assigned.values())
    rep.append(f"Total images: {total}")
    rep.append(f"Targets: {target_sizes}")
    rep.append("Global label counts: " + ", ".join([f"{l}:{global_counts[l]}" for l in all_labels]))
    rep.append("Global label ratios: " + ", ".join([f"{l}:{global_counts[l]/max(1,total):.3f}" for l in all_labels]))
    for s in ["train","val"]:  # , "test"
        if s not in assigned: 
            continue
        n = len(assigned[s])
        rep.append(f"\n[{s}] images: {n} (target {target_sizes[s]})")
        cnt = split_label_counts[s]
        rep.append("  label counts: " + ", ".join([f"{l}:{cnt[l]}" for l in all_labels]))
        rep.append("  label ratios: " + ", ".join([f"{l}:{cnt[l]/max(1,n):.3f}" for l in all_labels]))
        # 타깃 대비 라벨 분포 오차(참고용)
        errs = []
        for l in all_labels:
            tgt = label_targets[s][l]
            err = cnt[l] - tgt
            rel = 0.0 if tgt == 0 else err / max(1.0, tgt)
            errs.append(f"{l}:Δ{err:.1f}({rel:+.1%})")
        rep.append("  errors vs target: " + ", ".join(errs))
    return "\n".join(rep)

report = summarize()
print(report)
(Path(DST) / "split_report.txt").write_text(report, encoding="utf-8")
print(f"\n✅ Done. Split at: {DST}")
print("↳ data.yaml, split_report.txt saved.")


Total images: 405
Targets: {'train': 324, 'val': 81}
Global label counts: 0:224, 1:221, 2:88, 3:50, 4:0, 5:30, 6:50, 7:50, bg:0
Global label ratios: 0:0.553, 1:0.546, 2:0.217, 3:0.123, 4:0.000, 5:0.074, 6:0.123, 7:0.123, bg:0.000

[train] images: 324 (target 324)
  label counts: 0:174, 1:176, 2:70, 3:36, 4:0, 5:23, 6:39, 7:39, bg:0
  label ratios: 0:0.537, 1:0.543, 2:0.216, 3:0.111, 4:0.000, 5:0.071, 6:0.120, 7:0.120, bg:0.000
  errors vs target: 0:Δ-5.2(-2.9%), 1:Δ-0.8(-0.5%), 2:Δ-0.4(-0.6%), 3:Δ-4.0(-10.0%), 4:Δ0.0(+0.0%), 5:Δ-1.0(-4.2%), 6:Δ-1.0(-2.5%), 7:Δ-1.0(-2.5%), bg:Δ0.0(+0.0%)

[val] images: 81 (target 81)
  label counts: 0:50, 1:45, 2:18, 3:14, 4:0, 5:7, 6:11, 7:11, bg:0
  label ratios: 0:0.617, 1:0.556, 2:0.222, 3:0.173, 4:0.000, 5:0.086, 6:0.136, 7:0.136, bg:0.000
  errors vs target: 0:Δ5.2(+11.6%), 1:Δ0.8(+1.8%), 2:Δ0.4(+2.3%), 3:Δ4.0(+40.0%), 4:Δ0.0(+0.0%), 5:Δ1.0(+16.7%), 6:Δ1.0(+10.0%), 7:Δ1.0(+10.0%), bg:Δ0.0(+0.0%)

✅ Done. Split at: /home/dw/ws_job_msislab/Golf_Proj

In [10]:
from ultralytics import YOLO

# 데이터 설정: train/val 경로, 클래스 수/이름이 정확해야 함
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data_for_study_split/data.yaml"

# 모델 선택: s(가볍고 빠름) → 결과 좋아지면 m으로 확장
model = YOLO("yolov8s.pt")

results = model.train(
    # === 기본 학습 설정 ===
    data=DATA_YAML,      # 학습/검증 경로 정의된 data.yaml
    epochs=300,          # 총 학습 epoch (조기종료가 있으면 실제로는 더 적게 끝날 수 있음)
    patience=50,         # val metric이 개선 없을 때 조기종료 대기 epoch
    imgsz=1024,          # 입력 해상도(정사각). 소물체 ↑. OOM 시 896/768로
    batch=-1,            # 자동 배치 크기 탐색. OOM 나면 8~16으로 수동 고정
    device=0,            # GPU 인덱스 (여러 GPU면 "0,1" 형태 가능)
    workers=8,           # DataLoader 병렬 로드 워커 수 (디스크/CPU 상황에 맞춰)
    rect=True,           # aspect ratio 유지한 사각형 배치 → 소물체 왜곡↓, 수렴 안정화
    cache=True,          # 이미지/라벨 메모리 캐시. RAM 부족 시 False 권장
    amp=True,            # 자동 혼합정밀도(fp16). 최신 GPU에서 속도/메모리 이점

    # === 온라인 증강(가볍게) ===
    mosaic=0.10,         # 초반 모자이크 비율. 소물체 리콜↑. 과하면 artifacts↑
    close_mosaic=15,     # 지정 epoch 이후 mosaic=0 자동 전환(과적합 방지)
    mixup=0.0,           # MixUp 비활성(디봇/잔디 경계 왜곡 방지 목적)
    copy_paste=0.0,      # Copy-Paste 비활성(객체 경계/텍스처 왜곡 우려)
    hsv_h=0.01,          # Hue 변동(작게). 잔디 색 왜곡 최소화
    hsv_s=0.2,           # Saturation 변동
    hsv_v=0.2,           # Value(밝기) 변동
    degrees=0.0,         # 회전 금지(지형/그림자 방향성 유지 목적)
    translate=0.0,       # 이동 변형 금지(필요 시 0.05~0.1 권장)
    scale=0.0,           # 스케일 변형 금지(필요 시 0.10~0.20 권장)
    shear=0.0,           # 기울이기 금지(텍스처 왜곡 방지)
    erasing=0.0,         # 랜덤 Erasing 비활성(작은 객체 소실 방지)
    # ※ 오프라인 증강을 이미 다수 수행했다면 온라인은 약하게 유지하는 편이 안전

    # === 손실/최적화 쪽 튜닝 ===
    box=10.0,            # bbox 회귀 손실 가중(기본≈7.5). 위치 정밀도↑
    cls=0.5,             # 클래스 분류 손실 가중(기본 수준)
    dfl=1.5,             # Distribution Focal Loss 가중(경계 정밀도↑)
    # lr0/lrf: Cosine 스케줄(초기 lr → 최종 lr=lr0*lrf)
    lr0=0.002,           # 초기 lr (AdamW 기준 안전값 / SGD면 보통 0.01)
    lrf=0.01,            # 최종 lr 비율 (0.01이면 마지막에 lr0의 1%)
    momentum=0.9,        # SGD: 모멘텀, Adam/AdamW: beta1 역할로 사용됨(ultra 내부 호환)
    weight_decay=0.01,   # AdamW 권장: 0.01 / SGD 권장: 0.0005

    # === Optimizer 선택 ===
    # optimizer 인자를 생략하면 기본 SGD로 동작
    # optimizer="SGD",     # 대규모 데이터/보수적 일반화. 권장 세팅: lr0=0.01, wd=0.0005, momentum=0.937
    optimizer="AdamW",    # 소/중규모+증강 多에서 빠르고 안정적 수렴. 권장: lr0≈0.002, wd≈0.01
    # optimizer="Adam",    # 빠른 수렴(초반 성능↑)이나 과적합 주의. 권장: lr0≈0.001~0.002, wd 0~0.0005
    # optimizer="RMSProp", # 특수 케이스 외 권장X. 튜닝 민감.
    # optimizer="NAdam",   # Adam+Nesterov. 드물게 사용. 실험용.

    # === 출력 경로 ===
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",  # 상위 폴더
    name="20250904_data_study_1",  # 실험 이름(여러 실험 구분용)
    exist_ok=True            # 같은 이름 덮어쓰기 허용
)

Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=10.0, cache=True, cfg=None, classes=None, close_mosaic=15, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data_for_study_split/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.0, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.2, hsv_v=0.2, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.9, mosaic=0.1, multi_scale=False, name=20250904_data_study_1, nbs=64, nms=False, opset=None, optimize=False, opt

  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  8                  -1  1   1838080  ultralytics.nn.modules.block.C2f             [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    591360  ultralytics.nn.modules.block.C2f             [768, 256, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384,

train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data_for_study_split/labels/train.cache... 324 images, 0 backgrounds, 0 corrupt: 100%|██████████| 324/324 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=1024 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4060 Ti) 7.75G total, 2.79G reserved, 0.42G allocated, 4.53G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    11138696       73.37         2.108         9.155         13.82      (1, 3, 1024, 1024)                    list
    11138696       146.7         2.928            14         28.74      (2, 3, 1024, 1024)                    list
    11138696       293.5         4.039         30.44          59.4      (4, 3, 1024, 1024)                    list
    11138696         587         6.266         72.58         116.5      (8, 3, 1024, 1024)                    list
CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 7.75 GiB of which 21.56 MiB is free. Including non-PyTorch memory, this process has 7.34 GiB memory in use. Of the allocated memory 7.09 GiB is allocated by PyTorch, and 57.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables

...

    136/300      1.52G     0.2782     0.1701     0.7945          7       1024: 100%|██████████| 162/162 [00:07<00:00, 22.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:00<00:00, 34.14it/s]

                   all         81        235      0.883      0.821      0.883      0.674
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 86, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



136 epochs completed in 0.310 hours.
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250904_data_study_1/weights/last.pt, 22.6MB
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250904_data_study_1/weights/best.pt, 22.6MB

Validating /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250904_data_study_1/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,128,680 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:00<00:00, 33.56it/s]


                   all         81        235      0.915       0.81      0.904      0.691
                 Divot         50         79      0.962      0.634      0.905      0.717
           Fixed_Divot         45         76      0.981      0.763      0.946      0.796
         Dieased_Grass         18         37      0.758      0.432      0.557       0.39
                  Pole         14         14      0.974      0.929      0.963      0.645
             Sprinkler          7          7      0.985          1      0.995      0.908
                 Drain         11         11       0.83      0.909      0.968      0.758
             Golf ball         11         11      0.916          1      0.995      0.625
Speed: 0.4ms preprocess, 4.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250904_data_study_1
